In [ ]:
import os
from sklearn.metrics import confusion_matrix, classification_report
import cv2 as cv
import numpy as np

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
%cd FYP

/content/drive/MyDrive/FYP


In [ ]:
%ls

 Datasets/  'Saved Results'/  'Testing Datasets'/


In [ ]:
%cd 'Saved Results'

/content/drive/MyDrive/FYP/Saved Results


In [ ]:
%ls

ContrastImage_EfficientNetB4_20.h5
DenseNet121_20_10.h5
DenseNet121_20_10_retraining__epochs/
DenseNet121_20.h5
DenseNet121_training_20_epochs/
EfficientNetB4_20_10_10.h5
EfficientNetB4_20_10_10_retraining__epochs/
EfficientNetB4_20_10.h5
EfficientNetB4_20_10_retraining__epochs/
EfficientNetB4_20.h5
EfficientNetB4_20_retraining_10_epochs/
EfficientNetB4_training_20_epochs/
InceptionV3_20.h5
InceptionV3_training_20_epochs/
MobileNetV2_20.h5
MobileNetV2_training_20_epochs/
ResNet50_20_10.h5
ResNet50_20_10_retraining__epochs/
ResNet50_20.h5
ResNet50_training_20_epochs/


**Loading Models**

In [ ]:
EfficientNetB4 = load_model('EfficientNetB4_20_10_10.h5')

In [ ]:
# EfficientNetB4 = load_model('EfficientNetB4_20.h5')
# DenseNet121 = load_model('DenseNet121_20.h5')
# InceptionV3 = load_model('InceptionV3_20.h5')
# MobileNetV2 = load_model('MobileNetV2_20.h5')
# ResNet50 = load_model('ResNet50_20.h5')

**Loading Old models**

In [ ]:
%ls /content/drive/MyDrive/FYP_Saved_Models/

model_30_epochs.h5       training_50_epochs/
new_model_30_epochs.h5   VTumorDx_v5_10_epochs.h5
new_training_30_epochs/  VTumorDx_v5.2_10_epochs.h5
training_1/              VTumorDx_v5.2_training_10_epochs/
training_2/              VTumorDx_v5_training_10_epochs/
training_30_epochs/


In [ ]:
# mymodel = load_model("/content/drive/MyDrive/FYP_Saved_Models/VTumorDx_v5_10_epochs.h5")

**Loading Images**

In [ ]:
%cd /content/drive/MyDrive/FYP/Testing\ Datasets/

/content/drive/MyDrive/FYP/Testing Datasets


In [ ]:
%ls

Br35H_No/  KM/  MP/


In [ ]:
%ls Br35H_No/

no_tumor/


In [ ]:
%ls KM/Testing

glioma/  meningioma/  pituitary/


In [ ]:
%ls MP/Testing

glioma/  meningioma/  no_tumor/  pituitary/


In [ ]:
IMAGES = []
IMAGES_LABELS = []

In [ ]:
PATH = "/content/drive/MyDrive/FYP/Testing Datasets/"

PATHS = [
    "Br35H_No/",
    "KM/Training",
    "KM/Testing",
    "MP/Training",
    "MP/Testing"
]

In [ ]:
for p in PATHS:
  mypath = PATH + p
  for subdirectory in os.listdir(mypath):
    for images in os.listdir(mypath + "/" + subdirectory):
      img = cv.imread(mypath + "/" + subdirectory + "/" + images)
      img = cv.resize(img, (128, 128))
      IMAGES.append(img)
      IMAGES_LABELS.append(subdirectory)

In [ ]:
images, images_labels = np.array(IMAGES), np.array(IMAGES_LABELS)

In [ ]:
images.shape

(16354, 128, 128, 3)

In [ ]:
images_labels

array(['no_tumor', 'no_tumor', 'no_tumor', ..., 'meningioma',
       'meningioma', 'meningioma'], dtype='<U10')

In [ ]:
set(IMAGES_LABELS)

{'glioma', 'meningioma', 'no_tumor', 'pituitary'}

In [ ]:
CLASSES = ['glioma', 'meningioma', 'pituitary', 'no_tumor']

In [ ]:
import tensorflow as tf
temp_train = []
for i in images_labels:
    temp_train.append(CLASSES.index(i))
train_labels = temp_train
train_labels = tf.keras.utils.to_categorical(train_labels)

In [ ]:
images_labels

array(['no_tumor', 'no_tumor', 'no_tumor', ..., 'meningioma',
       'meningioma', 'meningioma'], dtype='<U10')

In [ ]:
train_labels

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
images_labels.shape

(16354,)

In [ ]:
import copy as cp

In [ ]:
images_labels = cp.deepcopy(train_labels)

In [ ]:
images_labels

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
def getModelResult(model, train_labels):
  loss, accuracy = model.evaluate(images, train_labels, verbose=2)
  print(loss, accuracy)
  Eff_pred = model.predict(images)

  train_labels = np.argmax(train_labels, axis=1)
  Eff_pred = np.argmax(Eff_pred, axis=1)

  print(classification_report(Eff_pred, train_labels))
  cm = confusion_matrix(Eff_pred, train_labels)
  print(cm)
  print(cm.diagonal()/cm.sum(axis=1))



In [ ]:
getModelResult(EfficientNetB4, images_labels)

512/512 - 38s - loss: 0.2647 - accuracy: 0.9508 - 38s/epoch - 75ms/step
0.26465219259262085 0.9508377313613892
512/512 [==============================] - 28s 49ms/step
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      4006
           1       0.97      0.90      0.94      4544
           2       0.92      1.00      0.96      4070
           3       0.98      0.92      0.95      3734

    accuracy                           0.95     16354
   macro avg       0.95      0.95      0.95     16354
weighted avg       0.95      0.95      0.95     16354

[[3937   30   17   22]
 [ 113 4111  268   52]
 [   0    4 4066    0]
 [ 138   82   78 3436]]
[0.98277584 0.90470951 0.9990172  0.92019282]


In [ ]:
getModelResult(EfficientNetB4, images_labels)

512/512 - 38s - loss: 0.1619 - accuracy: 0.9703 - 38s/epoch - 74ms/step
0.16192497313022614 0.9703436493873596
512/512 [==============================] - 27s 48ms/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      3957
           1       0.99      0.94      0.96      4445
           2       0.98      1.00      0.99      4367
           3       0.97      0.95      0.96      3585

    accuracy                           0.97     16354
   macro avg       0.97      0.97      0.97     16354
weighted avg       0.97      0.97      0.97     16354

[[3916    5   12   24]
 [ 149 4180   56   60]
 [   2    6 4353    6]
 [ 121   36    8 3420]]
[0.98963862 0.94038245 0.99679414 0.9539749 ]


In [ ]:
getModelResult(EfficientNetB4, train_labels)

512/512 - 41s - loss: 0.2560 - accuracy: 0.9484 - 41s/epoch - 81ms/step
0.25600388646125793 0.948391854763031
512/512 [==============================] - 31s 54ms/step
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      3761
           1       0.98      0.88      0.93      4696
           2       0.95      1.00      0.97      4195
           3       0.99      0.94      0.96      3702

    accuracy                           0.95     16354
   macro avg       0.95      0.95      0.95     16354
weighted avg       0.95      0.95      0.95     16354

[[3705   12   20   24]
 [ 379 4151  145   21]
 [   0    2 4191    2]
 [ 104   62   73 3463]]
[0.98511034 0.88394378 0.99904648 0.9354403 ]


In [ ]:
getModelResult(DenseNet121, images_labels)

512/512 - 22s - loss: 0.2256 - accuracy: 0.9496 - 22s/epoch - 42ms/step
0.22563698887825012 0.9496147632598877
512/512 [==============================] - 19s 33ms/step
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4241
           1       0.98      0.90      0.94      4607
           2       0.88      1.00      0.93      3903
           3       0.98      0.96      0.97      3603

    accuracy                           0.95     16354
   macro avg       0.95      0.95      0.95     16354
weighted avg       0.95      0.95      0.95     16354

[[4055   63   93   30]
 [  90 4130  367   20]
 [   0    2 3893    8]
 [  43   32   76 3452]]
[0.95614242 0.89646191 0.99743787 0.95809048]


In [ ]:
getModelResult(InceptionV3, images_labels)

512/512 - 17s - loss: 0.2297 - accuracy: 0.9461 - 17s/epoch - 34ms/step
0.22967784106731415 0.9460682272911072
512/512 [==============================] - 16s 27ms/step
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4087
           1       0.98      0.90      0.94      4590
           2       0.94      0.99      0.97      4233
           3       0.95      0.96      0.96      3444

    accuracy                           0.95     16354
   macro avg       0.95      0.95      0.95     16354
weighted avg       0.95      0.95      0.95     16354

[[3835   71  100   81]
 [ 232 4132  129   97]
 [  34    4 4184   11]
 [  87   20   16 3321]]
[0.93834108 0.90021786 0.98842429 0.96428571]


In [ ]:
getModelResult(ResNet50, images_labels)

512/512 - 20s - loss: 0.2713 - accuracy: 0.9365 - 20s/epoch - 39ms/step
0.2712867558002472 0.9364681243896484
512/512 [==============================] - 19s 34ms/step
              precision    recall  f1-score   support

           0       0.84      0.97      0.90      3615
           1       0.97      0.89      0.93      4577
           2       0.96      0.99      0.98      4302
           3       0.98      0.89      0.94      3860

    accuracy                           0.94     16354
   macro avg       0.94      0.94      0.94     16354
weighted avg       0.94      0.94      0.94     16354

[[3524   41   15   35]
 [ 349 4085  120   23]
 [  15   27 4257    3]
 [ 300   74   37 3449]]
[0.97482711 0.89250601 0.98953975 0.89352332]


In [ ]:
getModelResult(MobileNetV2, images_labels)

512/512 - 10s - loss: 0.4867 - accuracy: 0.8899 - 10s/epoch - 19ms/step
0.4867449998855591 0.8898740410804749
512/512 [==============================] - 8s 14ms/step
              precision    recall  f1-score   support

           0       0.75      0.97      0.85      3274
           1       0.91      0.97      0.94      3957
           2       0.92      1.00      0.96      4105
           3       0.99      0.69      0.81      5018

    accuracy                           0.89     16354
   macro avg       0.89      0.91      0.89     16354
weighted avg       0.91      0.89      0.89     16354

[[3161   90    7   16]
 [  50 3840   47   20]
 [   0    3 4090   12]
 [ 977  294  285 3462]]
[0.96548564 0.97043215 0.99634592 0.6899163 ]


In [ ]:
getModelResult(mymodel, images_labels)

512/512 - 39s - loss: 0.2262 - accuracy: 0.9547 - 39s/epoch - 76ms/step
0.22622326016426086 0.9546899795532227
512/512 [==============================] - 30s 52ms/step
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      3948
           1       0.99      0.89      0.94      4687
           2       0.96      1.00      0.98      4248
           3       0.94      0.95      0.95      3471

    accuracy                           0.95     16354
   macro avg       0.95      0.96      0.95     16354
weighted avg       0.96      0.95      0.95     16354

[[3878   12    6   52]
 [ 192 4187  164  144]
 [   0    6 4238    4]
 [ 118   22   21 3310]]
[0.9822695  0.89332195 0.99764595 0.95361567]


In [ ]:
%ls /content/drive/MyDrive/FYP_Saved_Models/

model_30_epochs.h5       training_50_epochs/
new_model_30_epochs.h5   VTumorDx_v5_10_epochs.h5
new_training_30_epochs/  VTumorDx_v5.2_10_epochs.h5
training_1/              VTumorDx_v5.2_training_10_epochs/
training_2/              VTumorDx_v5_training_10_epochs/
training_30_epochs/


In [ ]:
mymodel_30 = load_model("/content/drive/MyDrive/FYP_Saved_Models/model_30_epochs.h5")

In [ ]:
getModelResult(mymodel_30, images_labels)

512/512 - 33s - loss: 0.4859 - accuracy: 0.9288 - 33s/epoch - 64ms/step
0.4858655035495758 0.9287636280059814
512/512 [==============================] - 30s 53ms/step
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4196
           1       0.85      0.92      0.89      3933
           2       0.92      1.00      0.96      4096
           3       0.99      0.84      0.91      4129

    accuracy                           0.93     16354
   macro avg       0.93      0.93      0.93     16354
weighted avg       0.93      0.93      0.93     16354

[[4016   88   55   37]
 [  98 3612  223    0]
 [   0    8 4088    0]
 [  74  519   63 3473]]
[0.957102   0.91838291 0.99804688 0.84112376]


In [ ]:
mymodel_new_30 = load_model("/content/drive/MyDrive/FYP_Saved_Models/new_model_30_epochs.h5")

In [ ]:
getModelResult(mymodel_new_30, images_labels)

512/512 - 33s - loss: 0.5299 - accuracy: 0.9030 - 33s/epoch - 64ms/step
0.5298781394958496 0.9029595255851746
512/512 [==============================] - 30s 52ms/step
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      3544
           1       0.84      0.94      0.89      3755
           2       0.95      1.00      0.97      4206
           3       1.00      0.72      0.84      4849

    accuracy                           0.90     16354
   macro avg       0.91      0.91      0.90     16354
weighted avg       0.91      0.90      0.90     16354

[[3506   21   17    0]
 [ 102 3547  106    0]
 [   0    2 4204    0]
 [ 580  657  102 3510]]
[0.98927765 0.94460719 0.99952449 0.72386059]


In [ ]:
mymodel_20 = load_model("/content/drive/MyDrive/FYP_Saved_Models/VTumorDx_v5.2_10_epochs.h5")

In [ ]:
getModelResult(mymodel_20, images_labels)

512/512 - 30s - loss: 0.1664 - accuracy: 0.9711 - 30s/epoch - 58ms/step
0.16637548804283142 0.9711385369300842
512/512 [==============================] - 30s 52ms/step
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      4082
           1       0.99      0.94      0.96      4444
           2       0.98      0.99      0.99      4384
           3       0.96      0.98      0.97      3444

    accuracy                           0.97     16354
   macro avg       0.97      0.97      0.97     16354
weighted avg       0.97      0.97      0.97     16354

[[3990   24    0   68]
 [ 149 4173   75   47]
 [   1   11 4348   24]
 [  48   19    6 3371]]
[0.97746203 0.9390189  0.99178832 0.97880372]
